In [1]:
# install PyAudio http://people.csail.mit.edu/hubert/pyaudio/
import pyaudio
import wave
import sys
import requests

# default settings for record and playback
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 22050

def play_stream(wave_stream):
    global CHUNK
    print("Playing...")
    p = pyaudio.PyAudio()
    out_stream = p.open(format=p.get_format_from_width(wave_stream.getsampwidth()),
                channels=wave_stream.getnchannels(),
                rate=wave_stream.getframerate(),
                output=True)
    data = wave_stream.readframes(CHUNK)
    while data:
        out_stream.write(data)
        data = wave_stream.readframes(CHUNK)
    out_stream.stop_stream()
    out_stream.close()
    p.terminate()
    print("played")


def record_to_file(filename, seconds=5):
    global FORMAT, CHANNELS, RATE
    RECORD_SECONDS = seconds
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
    print("Start recording... ")
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print("...recorded", seconds, "second(s)")
    stream.stop_stream()
    stream.close()
    p.terminate()
    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [2]:
api_key = "" #TODO your key

In [3]:
def speak_out_yandex(query, dest, lang="en-US"):
    tts_url = "https://tts.voicetech.yandex.net/generate" + \
                "?key={}&text={}&format={}&quality={}&lang={}&speaker={}&speed=1.0&emotion={}"
    voice = "jane"
    tts_url = tts_url.format(api_key, query, "wav", "lo", lang, "jane", "evil")
    resp = requests.get(tts_url)
    with open(dest, "wb") as f:
        f.write(resp.content)

In [4]:
query = "How much is the fish"
tmp = "C:/dev/resp.wav"

speak_out_yandex(query, tmp)
play_stream(wave.open(tmp, 'rb'))

speak_out_yandex("Почём опиум для народа?", tmp, lang="ru-RU")
play_stream(wave.open(tmp, 'rb'))

Playing...
played
Playing...
played
